In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
import numpy as np

In [3]:
#https://towardsdatascience.com/multi-class-text-classification-with-scikit-learn-12f1e60e0a9f
%matplotlib inline
import pandas as pd
#mac
# df = pd.read_csv("/Users/ryanalcantara/Professional Drive/Subreddit/code_repo/literature_update/RYANDATA_trim.csv")
#pc
df = pd.read_csv(r'C:\Users\ryans\Pro Drive\Subreddit\code_repo\literature_update\RYANDATA_trim.csv')

col = ['topic','title']
df = df[col]
df = df[pd.notnull(df['title'])]

df.columns = ['topic','title']

df['topic_id'] = df['topic'].factorize()[0]
topic_id_df = df[['topic',
                  'topic_id']].drop_duplicates().sort_values('topic_id')
topic_to_id = dict(topic_id_df.values)
id_to_topic = dict(topic_id_df[['topic_id','topic']].values)
df.sample(5)

#histo of topics (ideally have >600 each)
import matplotlib.pyplot as plt
plt.rc('xtick',labelsize = 20)
plt.rc('ytick',labelsize = 20)
fig = plt.figure(figsize= (20,10))
df.groupby('topic').title.count().plot.bar(ylim=0)
plt.xlabel("paper topic",fontsize = 24)
plt.show()

#need to consolidate Pick of the Week papers (or just exclude them)

In [13]:
#model selection!

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.decomposition import LatentDirichletAllocation as LDA

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf = True,
                       min_df = 1,
                       norm = 'l2',
                       encoding = 'latin-1',
                       ngram_range = (1,2),
                       stop_words = 'english')
# https://scikit-learn.org/stable/modules/multiclass.html
features = tfidf.fit_transform(df.title).toarray()
labels = df.topic_id

models = [
    #RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0), #sucks
#     LinearSVC(multi_class='ovr', random_state=0), #one vs rest (all)
    LDA()
#     SGDClassifier(max_iter=500, random_state=0, tol = 1e-3),
#     PassiveAggressiveClassifier(max_iter = 500, random_state = 0, tol = 1e-3)
    #MultinomialNB(),
    #LogisticRegression(random_state=0, solver = 'liblinear', multi_class = 'ovr'),
]

In [8]:
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
plt.rc('xtick',labelsize = 12)
plt.rc('ytick',labelsize = 12)
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)

plt.xticks( rotation='25')

plt.show()

cv_df.groupby('model_name').accuracy.mean()

C:\Users\ryans\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


AttributeError: 'LatentDirichletAllocation' object has no attribute 'predict'

In [15]:
lda = LDA(n_components= 18, random_state=0)
lda.fit(features)


C:\Users\ryans\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=18, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [27]:
np.size(lda.components_,1)

64288

In [12]:
from sklearn.model_selection import train_test_split
model = LinearSVC()

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.20, random_state=0)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
plt.rc('xtick',labelsize = 20)
plt.rc('ytick',labelsize = 20)
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
fig.tight_layout(pad = 8)
sns.set(font_scale=1.2) #font size multiplier
sns.heatmap(conf_mat, annot=True, fmt='d', cmap = 'magma',
            xticklabels=topic_id_df.topic.values, yticklabels=topic_id_df.topic.values)

plt.ylabel('Actual',fontsize = 20)
plt.xlabel('Predicted',fontsize = 20)
plt.title('Number of Predicted vs. Actual Paper Topics', fontsize = 26)
plt.yticks( rotation='horizontal')

plt.savefig('biomchL_predict_plot.png')
plt.show()


C:\Users\ryans\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


AttributeError: 'LatentDirichletAllocation' object has no attribute 'predict'

In [28]:
from IPython.display import display

for predicted in topic_id_df.topic_id:
    for actual in topic_id_df.topic_id:
        if predicted != actual and conf_mat[actual, predicted] >= 2:
            print("'{}' predicted as '{}' : {} examples.".format(id_to_topic[actual], id_to_topic[predicted], conf_mat[actual, predicted]))
            display(df.loc[indices_test[(y_test == actual) & (y_pred == predicted)]][['topic', 'title']])
            print('')
            


'EVO/COMP' predicted as 'BONE' : 3 examples.


,topic,title
1917,EVO/COMP,Quantifying lateral femoral condyle elliptical...
1788,EVO/COMP,Nemaflex: a microfluidics-based technology for...
1783,EVO/COMP,Maintaining mobility after fracture: a biomech...



'HAND/FOOT' predicted as 'BONE' : 3 examples.


,topic,title
2103,HAND/FOOT,Deformable image registration and 3d strain ma...
2124,HAND/FOOT,Forces in the distal radius during a pushup or...
2325,HAND/FOOT,Bone shape difference between control and oste...



'HEAD' predicted as 'BONE' : 4 examples.


...

,topic,title
5040,ORTHOPAEDICS,The effect of coracoacromial ligament excision...
4890,ORTHOPAEDICS,Intraoperative preconditioning of fixed and ad...
4808,ORTHOPAEDICS,Shape and size of the medial patellofemoral li...



'REHABILITATION' predicted as 'TENDON/LIGAMENT' : 4 examples.


,topic,title
5969,REHABILITATION,Functional bracing of ACL injuries: current st...
5894,REHABILITATION,Subjective vs objective predictors of function...
5989,REHABILITATION,Jumping performance based on duration of rehab...
5981,REHABILITATION,Examination of knee morphology after secondary...



'SPORT/EXERCISE' predicted as 'TENDON/LIGAMENT' : 3 examples.


,topic,title
6452,SPORT/EXERCISE,Ankle dorsiflexion displacement is associated ...
6753,SPORT/EXERCISE,Achilles Tendon Biomechanics in Response to Ac...
6459,SPORT/EXERCISE,Clinical predictors of knee mechanics at retur...


In [27]:

from sklearn import metrics
print(metrics.classification_report(y_test, y_pred, target_names=df['topic'].unique()))

                 precision    recall  f1-score   support

           BONE       0.75      0.78      0.77        82
         CARDIO       0.78      0.78      0.78        78
       CELLULAR       0.72      0.78      0.75        74
     ERGONOMICS       0.72      0.81      0.76        77
       EVO/COMP       0.64      0.51      0.57        76
      HAND/FOOT       0.62      0.59      0.61        81
           HEAD       0.73      0.68      0.71        85
  IMPACT/TRAUMA       0.78      0.89      0.83        70
JOINT/CARTILAGE       0.71      0.61      0.66        88
       MODELING       0.58      0.43      0.50        81
         MUSCLE       0.65      0.74      0.69        77
         NEURAL       0.61      0.66      0.63        80
   ORTHOPAEDICS       0.63      0.58      0.60        78
POSTURE/BALANCE       0.67      0.90      0.76        67
 REHABILITATION       0.75      0.56      0.64       100
       ROBOTICS       0.79      0.89      0.84        74
 SPORT/EXERCISE       0.71    